In [103]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [19]:
df = pd.read_csv("https://raw.githubusercontent.com/Xiar-fatah/Stock-Market-Patterns/master/Core/MSFT.csv")

In [39]:
df = df.iloc[::-1]
df

,date,1. open,2. high,3. low,4. close,5. volume,SMA5,SMA10,SMA20,MOM6,MOM12,ULTOSC,RSI6,RSI12,WILLR,ROC
5003,2000-04-03,94.440,96.50,90.00,90.87,130059100.0,102.398,104.561,100.7820,-20.82,-4.50,32.4718,28.6321,39.0146,-96.5200,0.2759
5002,2000-04-04,91.560,92.00,84.94,88.56,90622200.0,99.248,103.142,100.5665,-15.50,-10.81,33.6782,26.2842,37.0450,-87.9574,-4.6409
5001,2000-04-05,88.250,88.50,85.87,86.37,41443800.0,95.084,101.454,100.1070,-17.94,-11.00,35.3427,24.0413,35.2067,-95.2428,-9.6170
5000,2000-04-06,87.870,88.00,85.27,86.00,33210700.0,91.610,98.867,99.4070,-21.19,-16.75,33.4155,23.6325,34.8876,-96.4737,-14.0000
4999,2000-04-07,87.000,89.37,85.00,89.06,41306800.0,88.172,96.604,98.8100,-14.31,-14.19,36.2056,34.6599,39.8093,-86.2941,-11.8218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2020-02-18,185.605,187.70,185.50,187.23,27853113.0,185.088,184.168,175.9405,3.34,14.45,59.4830,73.0306,72.9062,-11.3770,12.0467
3,2020-02-19,188.060,188.18,186.47,187.28,29997471.0,185.656,184.884,176.9795,-1.42,17.05,59.4591,73.1544,72.9627,-11.2131,12.4805
2,2020-02-20,186.950,187.25,181.10,184.42,36862376.0,185.598,185.336,177.9155,-0.02,10.04,53.4420,55.6272,64.5639,-20.5902,11.2975
1,2020-02-21,183.170,183.50,177.25,178.59,48600385.0,184.574,184.832,178.5090,-6.12,-1.53,53.2785,35.0722,51.4053,-39.7049,7.1197


In [40]:
input_ = df.iloc[:, [1,2,3,4,6,7,8,9,10,11,12,13,14,15]]
input_

,1. open,2. high,3. low,4. close,SMA5,SMA10,SMA20,MOM6,MOM12,ULTOSC,RSI6,RSI12,WILLR,ROC
5003,94.440,96.50,90.00,90.87,102.398,104.561,100.7820,-20.82,-4.50,32.4718,28.6321,39.0146,-96.5200,0.2759
5002,91.560,92.00,84.94,88.56,99.248,103.142,100.5665,-15.50,-10.81,33.6782,26.2842,37.0450,-87.9574,-4.6409
5001,88.250,88.50,85.87,86.37,95.084,101.454,100.1070,-17.94,-11.00,35.3427,24.0413,35.2067,-95.2428,-9.6170
5000,87.870,88.00,85.27,86.00,91.610,98.867,99.4070,-21.19,-16.75,33.4155,23.6325,34.8876,-96.4737,-14.0000
4999,87.000,89.37,85.00,89.06,88.172,96.604,98.8100,-14.31,-14.19,36.2056,34.6599,39.8093,-86.2941,-11.8218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,185.605,187.70,185.50,187.23,185.088,184.168,175.9405,3.34,14.45,59.4830,73.0306,72.9062,-11.3770,12.0467
3,188.060,188.18,186.47,187.28,185.656,184.884,176.9795,-1.42,17.05,59.4591,73.1544,72.9627,-11.2131,12.4805
2,186.950,187.25,181.10,184.42,185.598,185.336,177.9155,-0.02,10.04,53.4420,55.6272,64.5639,-20.5902,11.2975
1,183.170,183.50,177.25,178.59,184.574,184.832,178.5090,-6.12,-1.53,53.2785,35.0722,51.4053,-39.7049,7.1197


In [41]:
data = np.array(input_)
data

array([[ 94.44  ,  96.5   ,  90.    , ...,  39.0146, -96.52  ,   0.2759],
       [ 91.56  ,  92.    ,  84.94  , ...,  37.045 , -87.9574,  -4.6409],
       [ 88.25  ,  88.5   ,  85.87  , ...,  35.2067, -95.2428,  -9.617 ],
       ...,
       [186.95  , 187.25  , 181.1   , ...,  64.5639, -20.5902,  11.2975],
       [183.17  , 183.5   , 177.25  , ...,  51.4053, -39.7049,   7.1197],
       [167.77  , 174.55  , 163.23  , ...,  39.7366, -64.9508,   3.5446]])

In [104]:
def dataLoader(data, seq_lenght):
    #Sliding window concept
    N = len(data)
    labeled_data = []
    
    for i in range(seq_lenght, N):
        x = data[i-seq_lenght:i] 
        y = data[i, 3]
        
        labeled_data.append((x, y))
        
    return np.array(labeled_data)

structured_data = dataLoader(data,30)
assert structured_data[-1][0][-3,3] == 187.28

In [112]:
# Number of input features for the LSTM
num_features = structured_data[0][0].shape[1]

# Sequence lenght of input
seq_lenght = structured_data[0][0].shape[0]

assert num_features == 14
assert seq_lenght == 30

In [114]:
structured_data.shape

(4974, 2)

# LSTM

## Create LSTM model.

#### Build LSTM Network
Understand the structure of pytorch LSTM Network

In [116]:
class LSTM(nn.Module):
    def __init(self, input_shape, output_shape, hidden_shape):
        super(LSTM, self).__init__()
        
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.hidden_shape = hidden_shape
        
        self.h = torch.randn(1,1,1)
        self.c = torch.randn(1,1,1)
        
        
    def forward(self, t):
        return t

In [142]:
rnn = nn.LSTM(10, 20, 1)

In [154]:
input = torch.randn(5, 3, 10)
h0 = torch.randn(1, 3, 20)
c0 = torch.randn(1, 3, 20)

In [155]:
output, (hn, cn) = rnn(input)

In [157]:
output.shape

torch.Size([5, 3, 20])

#### Random sampling from sequence to make data uncorrolated.
Sample from sequenced-structured data, to see if the model becomes less reliant on sequential data.

#### Tune hyper parameters
Find hyper parameters that best maximizes the results.
Compare model with MAPE, MSE, RMS

## Write report
Plot and write first draft of project report.